# Qiskit nature test

In this test we will use a fixed distance for the bond, in order to get just one eigenvalue. The real bond energy plot is obtained by iterating over the bond length and saving the ground state eigenenergy. Interpolating them, we get the Lennard-Jones curve. This calculation was perfomed for different optimizers (see final cells to see what this means) and different bases.

## Definition of the system

Defining all the imports

In [40]:
import qiskit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit_nature.second_q.operators import electronic_integrals
from qiskit_nature.units import DistanceUnit

Note that in the following code we will define the PySCFDriver. **PySCF** is the library for classical chemistry computation, but now we need an interface between this library and qiskit: because of this, PySCFDriver is used to define all necessary data for computation of how electrons behave, their energy structure and so on.

The basis argument, insteand, simply is the set of functions used to model the electron wave function in the specific approximation we are (Hartree-Fock or whatever). _sto3g_ stands for Slater-type orbital with 3 Gaussian functions. It models the orbitals starting from 3 gaussians. It is a simple way to model orbitals, but it is not enough to obtain publication-level results. Visit https://downloads.wavefun.com/FAQ/BasisSetFAQ.html for more details

In [41]:
# defining the molecular coordinates (aka spatial position in angstroms)
distance_unit = DistanceUnit.ANGSTROM
driver = PySCFDriver(
    atom = 'Li .0 .0 .0; H 3.0 .0 .0',
    # atom = 'Li .0 .0 .0; H 1.6 .0 .0',
    basis = 'sto3g'
)

qmolecule = driver.run()
print(qmolecule)

print(qiskit.__version__)

1.4.2


In [42]:
# collecting Hamiltonian related to the molecule
electronic_energy: ElectronicEnergy = qmolecule.hamiltonian

# calculating the energy configuration of the molecule: one_body = electrons are independent
h1 = electronic_energy.electronic_integrals.one_body
# calculating the energy configuration of the molecule: two_body = electron-electron interaction is considered
h2 = electronic_energy.electronic_integrals.two_body

print(h1, h2)

<qiskit_nature.second_q.operators.electronic_integrals.ElectronicIntegrals object at 0x7f2c7c737ec0> <qiskit_nature.second_q.operators.electronic_integrals.ElectronicIntegrals object at 0x7f2cee16c3e0>


In [43]:
# collecting nuclear repulsion energy
nuclear_repulsion_energy = electronic_energy.nuclear_repulsion_energy

print(nuclear_repulsion_energy)

0.52917721092



Now we will _re_-define the Hamiltonian by means of a fermionic operator: this is due to the fact that calcualtions are easier if the notation used is the one of the second quantization, i.e. everything is expressed as **creation** and **annihilation** operators instead of position and momentum.

Here we use the _electronic_energy_ variable, which simply is the Hamiltonian associated to the molecule

In [44]:
ferm_op = electronic_energy.second_q_op()

print(ferm_op)

Fermionic Operator
number spin orbitals=12, number terms=1860
  -4.573998049690632 * ( +_0 -_0 )
+ 0.10284404056048234 * ( +_0 -_1 )
+ 0.154908586326399 * ( +_0 -_2 )
+ 0.038157654935360316 * ( +_0 -_5 )
+ 0.10284404056048231 * ( +_1 -_0 )
+ -1.1066142885458115 * ( +_1 -_1 )
+ -0.029677165681516304 * ( +_1 -_2 )
+ -0.08434934694893029 * ( +_1 -_5 )
+ 0.15490858632639912 * ( +_2 -_0 )
+ -0.02967716568151632 * ( +_2 -_1 )
+ -1.0495781505549477 * ( +_2 -_2 )
+ -0.00032233536092881243 * ( +_2 -_5 )
+ -1.0411793562389415 * ( +_3 -_3 )
+ -1.0411793562389415 * ( +_4 -_4 )
+ 0.0381576549353603 * ( +_5 -_0 )
+ -0.08434934694893038 * ( +_5 -_1 )
+ -0.0003223353609288208 * ( +_5 -_2 )
+ -1.0158151870325394 * ( +_5 -_5 )
+ -4.573998049690632 * ( +_6 -_6 )
+ 0.10284404056048234 * ( +_6 -_7 )
+ 0.154908586326399 * ( +_6 -_8 )
+ 0.038157654935360316 * ( +_6 -_11 )
+ 0.10284404056048231 * ( +_7 -_6 )
+ -1.1066142885458115 * ( +_7 -_7 )
+ -0.029677165681516304 * ( +_7 -_8 )
+ -0.08434934694893029 * ( +

Now calculating the total particles. We could in principle just use the *.num_particles* method of *qmolecule*, but it is a tuple, where the elements are respectively the electrons differentiated wrt spin (alpha = up, beta = down).

In [45]:
num_particles = sum(el for el in qmolecule.num_particles)

print(num_particles)
print(qmolecule.num_alpha, qmolecule.num_beta)
print(qmolecule.num_particles)

4
2 2
(2, 2)


As we can see from previous cells, the total number of spin orbitals associated to the molecule are higher than the ones we really need. Because of that we either freeze or remove some of them using **ActiveSpaceTransformer** from **second_q.transformers**. It takes as input the number of electrons and of molecular orbitals we want to keep/expect. The result is stored inside a new variable that will have a reduced number of orbitals, i.e. the one we will use as definitive system for the analysis

In [46]:
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

transformer = ActiveSpaceTransformer(num_electrons = 2, num_spatial_orbitals = 2)
molecule_reduced = transformer.transform(qmolecule)

print(f"Previous number of spin orbitals: {qmolecule.num_spin_orbitals}")
print(f"New number of spin orbitals: {molecule_reduced.num_spin_orbitals}")
print(f"Number of erased spin orbitals: {qmolecule.num_spin_orbitals - molecule_reduced.num_spin_orbitals}")

Previous number of spin orbitals: 12
New number of spin orbitals: 4
Number of erased spin orbitals: 8


Now we must update the fermionic operator (we will define a new one)

In [47]:
# collecting Hamiltonian related to the molecule
ferop = molecule_reduced.hamiltonian.second_q_op()

print(ferop)

Fermionic Operator
number spin orbitals=4, number terms=72
  -0.576466430427333 * ( +_0 -_0 )
+ 0.08953337662434449 * ( +_0 -_1 )
+ 0.08953337662434449 * ( +_1 -_0 )
+ -0.3364804426116338 * ( +_1 -_1 )
+ -0.576466430427333 * ( +_2 -_2 )
+ 0.08953337662434449 * ( +_2 -_3 )
+ 0.08953337662434449 * ( +_3 -_2 )
+ -0.3364804426116338 * ( +_3 -_3 )
+ 0.20048977283700814 * ( +_0 +_0 -_0 -_0 )
+ -0.04476669340067061 * ( +_0 +_0 -_1 -_0 )
+ -0.04476669340067061 * ( +_0 +_1 -_0 -_0 )
+ 0.11368503040128874 * ( +_0 +_1 -_1 -_0 )
+ 0.20048977283700814 * ( +_0 +_2 -_2 -_0 )
+ -0.04476669340067061 * ( +_0 +_2 -_3 -_0 )
+ -0.04476669340067061 * ( +_0 +_3 -_2 -_0 )
+ 0.11368503040128874 * ( +_0 +_3 -_3 -_0 )
+ -0.04476669340067061 * ( +_0 +_0 -_0 -_1 )
+ 0.03051515549188602 * ( +_0 +_0 -_1 -_1 )
+ 0.03051515549188602 * ( +_0 +_1 -_0 -_1 )
+ 0.007326848916608043 * ( +_0 +_1 -_1 -_1 )
+ -0.04476669340067061 * ( +_0 +_2 -_2 -_1 )
+ 0.03051515549188602 * ( +_0 +_2 -_3 -_1 )
+ 0.03051515549188602 * ( +_0 +_

## Definition of the qubit problem (parity mapper)

Now we need to map our model into a qubit problem, so that quantum computing can be used. By doing so the library applies the so called *two_qubit_reduction*, which simplifies the system by exploiting the $Z_2$ symmetries, lowering the computational load (in order to implement it, we need at least $num\_spin\_orbitals = 4$). The latter a symmetry such that the eigenvalues of the system do not change after 180-degrees rotation across an axis. In other words, if the system is described by an Hamiltonian which can be simplified by grouping the action of an operator, then the final result will only differ from the original one by means of a sign.

Without such reduction, we would have
$$num\_qubit = num\_spin\_orbitals$$

In order to translate our fermionic problem into a qubit problem, we need to use a **mapper**. The mappers taht can be used to perform a fermionic-to-qubit translation are **ParityMapper**, **JordanWignerMapper** and **BravyKitaevMapper**.

In [48]:
from qiskit_nature.second_q.mappers import ParityMapper

mapper = ParityMapper(num_particles = molecule_reduced.num_particles)

qubit_op = mapper.map(ferop)

print(qubit_op)
print(f"Qubits used: {qubit_op.num_qubits}")

SparsePauliOp(['II', 'IZ', 'IX', 'ZI', 'XI', 'ZZ', 'XZ', 'ZX', 'XX'],
              coeffs=[-0.62501416+0.j,  0.09375091+0.j,  0.05209353+0.j, -0.09375091+0.j,
  0.05209353+0.j, -0.06056266+0.j,  0.05209354+0.j, -0.05209354+0.j,
  0.06103031+0.j])
Qubits used: 2


Now we need to compute the eigenvalues of the qubit operator. There are two options: the first one uses **NumPyEigensolver**, which calculates exactly the eigenvalues with matrix calculations. The other option is to use the **minimum_eigensolver**, which can be applied at any desired quantum algorithm to compute the eigenvalues. In other words, one find the real eigenvalues with NumPyEigensolver to have a reference, then, once the solver is defined (e.g. VQE), we can compute the eigenvalues and compare the results.

In [49]:
from qiskit_algorithms.eigensolvers import NumPyEigensolver
from qiskit_algorithms.minimum_eigensolvers import numpy_minimum_eigensolver

num_eigenval = 5
solver = NumPyEigensolver(num_eigenval)
result = solver.compute_eigenvalues(qubit_op)

print(f"Classical result: {result}")
print(f"Eigenvalues: {result.eigenvalues}")

Classical result: {   'aux_operators_evaluated': None,
    'eigenstates': [   Statevector([ 0.1748068 +0.j,  0.93878422+0.j, -0.23993615+0.j,
              0.1748068 +0.j],
            dims=(2, 2)),
                       Statevector([ 7.07106781e-01+0.j,  5.21804822e-15+0.j, -5.55111512e-16+0.j,
             -7.07106781e-01+0.j],
            dims=(2, 2)),
                       Statevector([-0.62019036+0.j,  0.32205141+0.j,  0.35638563+0.j,
             -0.62019036+0.j],
            dims=(2, 2)),
                       Statevector([0.29121555-0.j, 0.12234002+0.j, 0.90300605+0.j,
             0.29121555+0.j],
            dims=(2, 2))],
    'eigenvalues': array([-0.76755156, -0.74660712, -0.68441646, -0.30148148])}
Eigenvalues: [-0.76755156 -0.74660712 -0.68441646 -0.30148148]


Now we implement the the second option. In order to do so, we need to import the **HartreeFock** module, which generates the quantum circuit associated to the reduced molecule 

In [50]:
from qiskit_nature.second_q.circuit.library.initial_states import HartreeFock

hf_circ = HartreeFock(
    num_spatial_orbitals = molecule_reduced.num_spatial_orbitals,
    num_particles        = molecule_reduced.num_particles,
    qubit_mapper         = mapper
)
print(hf_circ)

     ┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────
          


Now that the initial state is ready, we can import **UCSSD** in order to define the ansatz

In [51]:
from qiskit_nature.second_q.circuit.library.ansatzes import UCCSD

UCCSD_var_form = UCCSD(
    num_spatial_orbitals = molecule_reduced.num_spatial_orbitals,
    num_particles        = molecule_reduced.num_particles,
    qubit_mapper         = mapper,
    initial_state        = hf_circ,
    reps                 = 2
)
# print(UCCSD_var_form)

Now we have all the ingredients in order to define the **Variational Quantum Eigensolver** (**VQE**). In order to implement this algorithm, we need an estimator to compute the expectation value and a classical optimizer in order to define the values for the next iteration

In [ ]:
from qiskit_algorithms.optimizers import SLSQP, GradientDescent, ADAM, CG
from qiskit.primitives import Estimator, StatevectorEstimator
from qiskit_algorithms import VQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

optimizers = [
    CG(),
    SLSQP(),
    ADAM()
]
optimizers_str = [
    'CG',
    'SLSQP',
    'ADAM'
]

estimator = Estimator()
# estimator = StatevectorEstimator()


for idx, el in enumerate(optimizers):
    vqe_solver = VQE(
        estimator = estimator,
        optimizer = el,
        ansatz = UCCSD_var_form
    )

    vqe_gs_solver = GroundStateEigensolver(
        qubit_mapper = mapper,
        solver = vqe_solver
    )

    vqe_result = vqe_gs_solver.solve(molecule_reduced)
    print(f"Ground state eigenvalue with optimizer {optimizers_str[idx]}: {vqe_result.eigenvalues[0]}")
    print(f"Ground state total energy: {vqe_result.total_energies[0]}")

Ground state eigenvalue with optimizer GradientDescent: -0.7665342574016656
Ground state total energy: -7.7254108424382455
Ground state eigenvalue with optimizer SLSQP: -0.7675515550777655
Ground state total energy: -7.726428140114345
Ground state eigenvalue with optimizer ADAM: -0.7675515552100481
Ground state total energy: -7.726428140246627


Since it takes a while, we can parallelize the process using **ProcessPoolExecutor**, which allows parallelize (with respect to the optimizers) the simulation on different processors 

In [ ]:
import concurrent.futures

def run_vqe(opt_idx):
    el = optimizers[opt_idx]
    vqe_solver = VQE(
        estimator = estimator,
        optimizer = el,
        ansatz = UCCSD_var_form
    )
    vqe_gs_solver = GroundStateEigensolver(
        qubit_mapper = mapper,
        solver = vqe_solver
    )
    vqe_result = vqe_gs_solver.solve(molecule_reduced)
    return (optimizers_str[opt_idx], vqe_result.total_energies[0])

with concurrent.futures.ProcessPoolExecutor() as executor:
    futures = [executor.submit(run_vqe, idx) for idx in range(len(optimizers))]
    for future in concurrent.futures.as_completed(futures):
        key, value = future.result()
        print(f"{key}: {value}")


SLSQP: -0.7675515513909144
GradientDescent: -0.767148979623988
ADAM: -0.767551555210843


## Definition of the qubit problem (Jordan-Wigner mapper)

In this case, we implement the Jordan-Wigner mapper, which does not support the $Z_2$ symmetries reduction, thus it will take longer to simulate. For further reading, https://pennylane.ai/qml/demos/tutorial_mapping

In [ ]:
from qiskit_nature.second_q.mappers import JordanWignerMapper

mapper = JordanWignerMapper()

qubit_op = mapper.map(ferop)

print(qubit_op)
print(f"Qubits used: {qubit_op.num_qubits}")

num_eigenval = 5
solver = NumPyEigensolver(num_eigenval)
result = solver.compute_eigenvalues(qubit_op)

hf_circ = HartreeFock(
    num_spatial_orbitals = molecule_reduced.num_spatial_orbitals,
    num_particles        = molecule_reduced.num_particles,
    qubit_mapper         = mapper
)

UCCSD_var_form = UCCSD(
    num_spatial_orbitals = molecule_reduced.num_spatial_orbitals,
    num_particles        = molecule_reduced.num_particles,
    qubit_mapper         = mapper,
    initial_state        = hf_circ,
    reps                 = 2
)

optimizers = [
    CG(),
    SLSQP(),
    ADAM()
]
optimizers_str = [
    'CG',
    'SLSQP',
    'ADAM'
]

estimator = Estimator()
# estimator = StatevectorEstimator()


for idx, el in enumerate(optimizers):
    vqe_solver = VQE(
        estimator = estimator,
        optimizer = el,
        ansatz = UCCSD_var_form
    )

    vqe_gs_solver = GroundStateEigensolver(
        qubit_mapper = mapper,
        solver = vqe_solver
    )

    vqe_result = vqe_gs_solver.solve(molecule_reduced)
    print(f"Ground state eigenvalue with optimizer {optimizers_str[idx]}: {vqe_result.eigenvalues[0]}")
    print(f"Ground state total energy: {vqe_result.total_energies[0]}")